# Sentiment Analysis on Twitter: Deep Learning Approach 2

*   List item
*   List item


Author: Niloy Farhan

Model Description:

Glove Embedding Layer -> Bidirectional LSTM Layer -> Dense(300) Layer -> Softmax Activation Layer

Best Score after pre-processing (Glove.840B.300D):

81% Accuracy

# Imports

In [ ]:
import numpy as np 
import pandas as pd 
import re
import string
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import layers
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, GRU
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.utils import resample
import nltk
from nltk.corpus import stopwords


In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# TPU INITIALIZATION

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.8.0
Running on TPU  ['10.29.205.210:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.29.205.210:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.29.205.210:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Dataset Loading

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2013dev-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df1 = df1.drop(labels=["id"], axis = 1)
df2 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2013test-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df2 = df2.drop(labels=["id"], axis = 1)
df3 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2013train-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df3 = df3.drop(labels=["id"], axis = 1)
df4 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2014sarcasm-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df4 = df4.drop(labels=["id"], axis = 1)
df5 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2015test-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df5 = df5.drop(labels=["id"], axis = 1)
df6 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2015train-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df6 = df6.drop(labels=["id"], axis = 1)
df7 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2016dev-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df7 = df7.drop(labels=["id"], axis = 1)
df8 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2016devtest-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df8 = df8.drop(labels=["id"], axis = 1)
df9 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2016train-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df9 = df9.drop(labels=["id"], axis = 1)

In [ ]:
# df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9]).reset_index(drop=True)
df = pd.concat([df8,df9]).reset_index(drop=True)

df

,sentiment,tweets
0,neutral,@SeeMonterey LOST - Sony cell phone with holid...
1,neutral,"@PersonaSoda well yeah, that's third parties. ..."
2,negative,Sony rewards app is like a lot of 19 y.o femal...
3,neutral,@fakethom Have android tab and don't use phone...
4,positive,Finally I get my ps4 back I sent it to Sony ca...
...,...,...
7863,positive,@Racalto_SK ok good to know. Punting at MetLif...
7864,neutral,everyone who sat around me at metlife was so a...
7865,neutral,what giants or niners fans would wanna go to t...
7866,positive,Anybody want a ticket for tomorrow Colombia vs...


# Balancing the dataset

Our dataset is unbalanced when it comes to negative sentiment. So, we will resample to balance it.

In [ ]:
df['sentiment'].value_counts()

positive    4011
neutral     2682
negative    1175
Name: sentiment, dtype: int64

In [ ]:
def dataset_balance(data):
    data_positive = data[(data['sentiment']=='positive')]
    data_neutral = data[(data['sentiment']=='neutral')]
    data_negative = data[(data['sentiment']=='negative')]
    # data['sentiment'].value_counts()[2]
    # data_positive_downsampled = resample(data_positive, replace = True, n_samples = 6000, random_state = 42)
    data_neutral_downsampled = resample(data_neutral, replace = True, n_samples =  data['sentiment'].value_counts()[0], random_state = 42)
    data_negative_upsampled = resample(data_negative, replace = True, n_samples =  data['sentiment'].value_counts()[0], random_state = 42)
    balanced_dataset = pd.concat([data_positive,data_neutral_downsampled,data_negative_upsampled]).reset_index(drop=True)
    return balanced_dataset

In [ ]:
df = dataset_balance(df)
df['sentiment'].value_counts()

positive    4011
neutral     4011
negative    4011
Name: sentiment, dtype: int64

# PRE-PROCESSING

1. removed link
2. removed linebreaks
3. removed extra spaces
4. removed punctuation
5. removed stopwords
6. lowercased the tweet
7. extracted hashtags and added at the end of the sentence
8. extracted mentions and added at the end of the sentence

In [ ]:
def label_encoder(x):
    if x == 'positive': return 2
    elif x == 'negative': return 0
    else: return 1

def clean_text(x):
    x = re.sub(r'https?://\S+', '', x) 
    x = re.sub(r'#\w+', '', x) 
    x = re.sub(r'@\w+', '', x) 
    x = re.sub(r'\n',' ',x) 
    x = re.sub('\s+', ' ', x).strip()
    x = re.sub('\.','',x) 
    for p in string.punctuation:
        x = re.sub('\{}'.format(p),'',x)
    return x.lower()
def find_hashtags(tweet):
    return " ".join([match.group(0)[1:] for match in re.finditer(r"#\w+", tweet)]) or ''

def find_mentions(tweet):
    return " ".join([match.group(0)[1:] for match in re.finditer(r"@\w+", tweet)]) or ''

def pre_process_text(df):
    stop = stopwords.words('english')
    df['clean_tweet'] = df['tweets'].apply(lambda x: clean_text(x))
    df['hashtags'] = df['tweets'].apply(lambda x: find_hashtags(x))
    df['mentions'] = df['tweets'].apply(lambda x: find_mentions(x))
    df['sentiment'] = df['sentiment'].apply(lambda x: label_encoder(x))
    df['clean_tweet_without_stopword'] = df['clean_tweet'].apply(lambda x:' '.join([word for word in x.split() if word not in stop]))
    df['final'] = df['clean_tweet_without_stopword']+" "+df['hashtags']+" "+df['mentions']
    # df['clean_tweet']+" "+
    return df


In [ ]:
final_dataset = pre_process_text(df)

In [ ]:
final_dataset.tail()

,sentiment,tweets,clean_tweet,hashtags,mentions,clean_tweet_without_stopword,final
12028,0,Why would you get married on Sept 11th?! If yo...,why would you get married on sept 11th if you ...,,,would get married sept 11th want special date ...,would get married sept 11th want special date ...
12029,0,May all the items saved for later in your Amaz...,may all the items saved for later in your amaz...,moderncurses,,may items saved later amazon basket increase p...,may items saved later amazon basket increase p...
12030,0,"ok so i want to cry like, my expected delivery...",ok so i want to cry like my expected delivery ...,,,ok want cry like expected delivery date ipad t...,ok want cry like expected delivery date ipad t...
12031,0,De Rossi believes Juventus may struggle to cop...,de rossi believes juventus may struggle to cop...,,,de rossi believes juventus may struggle cope s...,de rossi believes juventus may struggle cope s...
12032,0,ICYMI: The 'failure' of Google+: A round-up of...,icymi the failure of google a roundup of inter...,,,icymi failure google roundup interesting techn...,icymi failure google roundup interesting techn...


Longest sentence

In [ ]:
final_dataset['final'].apply(lambda x:len(str(x).split())).max()

689

In [ ]:
Y = pd.get_dummies(final_dataset['sentiment']).values
print('Shape of label tensor:', Y.shape)
X = final_dataset['final']

Shape of label tensor: (12033, 3)


# Data spliting

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=.8, random_state=42, shuffle = True)

# Tokenizing, Padding, Embedding

In [ ]:
maxlen = 800
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1  

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)

X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
def create_embedding_matrix(filepath, word_idx, embedding_dim):
    vocab_size = len(word_idx) + 1  
    emb = np.zeros((vocab_size, embedding_dim))
    with open(filepath, 'r', encoding="utf8") as f:
        for i in f:
            x = i.split(' ')
            w, v = x[0], x[1:]
            if w in word_idx:
                emb[word_idx[w]] = np.asarray(v)[:embedding_dim]
    return emb

In [ ]:
embedding_dim = 300
embedding_matrix = create_embedding_matrix('/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/glove.840B.300d.txt', tokenizer.word_index, embedding_dim)

# My Model

In [ ]:
with tpu_strategy.scope():
  model = Sequential()
  model.add(layers.Embedding(vocab_size, embedding_dim, 
                            weights=[embedding_matrix], 
                            input_length=maxlen, 
                            trainable=False))
  model.add(Bidirectional(LSTM(300,activation='tanh', dropout=0.2, recurrent_dropout=0.2)))
  model.add(Dense(300,activation = 'relu'))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 800, 300)          4982100   
                                                                 
 bidirectional (Bidirectiona  (None, 600)              1442400   
 l)                                                              
                                                                 
 dense (Dense)               (None, 300)               180300    
                                                                 
 dense_1 (Dense)             (None, 3)                 903       
                                                                 
Total params: 6,605,703
Trainable params: 1,623,603
Non-trainable params: 4,982,100
_________________________________________________________________


In [ ]:
Y_train.shape

(9626, 3)

In [ ]:
epochs = 25
batch_size = 16*tpu_strategy.num_replicas_in_sync
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=2, min_delta=0.0001)])

Epoch 1/25
68/68 [==============================] - 44s 471ms/step - loss: 0.9099 - accuracy: 0.5572 - val_loss: 0.8561 - val_accuracy: 0.5909
Epoch 2/25
68/68 [==============================] - 24s 359ms/step - loss: 0.7791 - accuracy: 0.6494 - val_loss: 0.8206 - val_accuracy: 0.6220
Epoch 3/25
68/68 [==============================] - 24s 359ms/step - loss: 0.6822 - accuracy: 0.6935 - val_loss: 0.7596 - val_accuracy: 0.6739
Epoch 4/25
68/68 [==============================] - 24s 358ms/step - loss: 0.5675 - accuracy: 0.7630 - val_loss: 0.6453 - val_accuracy: 0.7165
Epoch 5/25
68/68 [==============================] - 24s 358ms/step - loss: 0.4564 - accuracy: 0.8146 - val_loss: 0.5891 - val_accuracy: 0.7591
Epoch 6/25
68/68 [==============================] - 24s 358ms/step - loss: 0.3595 - accuracy: 0.8549 - val_loss: 0.6130 - val_accuracy: 0.7664
Epoch 7/25
68/68 [==============================] - 24s 359ms/step - loss: 0.2831 - accuracy: 0.8916 - val_loss: 0.5773 - val_accuracy: 0.7934

# Test Data score

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

76/76 [==============================] - 8s 92ms/step - loss: 0.5904 - accuracy: 0.8168
Test set
  Loss: 0.590
  Accuracy: 0.817
